In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch-lightning
!pip install wandb
!wandb login

In [3]:
import torch

torch.cuda.is_available()

True

In [ ]:
!git clone https://github.com/xpwu95/LDL

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import shutil

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#links to two archives with database
# https://drive.google.com/file/d/1ysutxP3IHKhQA8brdwPlhwbduo-kYt1x/view?usp=sharing
# https://drive.google.com/file/d/1NqsL5mlI3iTsPmjvVX7y6GUileM7oL5v/view?usp=sharing

fileDownloaded = drive.CreateFile({'id':'1ysutxP3IHKhQA8brdwPlhwbduo-kYt1x'})
fileDownloaded.GetContentFile('Classification.tar')

fileDownloaded = drive.CreateFile({'id':'1NqsL5mlI3iTsPmjvVX7y6GUileM7oL5v'})
fileDownloaded.GetContentFile('Detection.tar')

shutil.unpack_archive('/content/drive/MyDrive/openface_test_task/Classification.tar', format='tar')
shutil.unpack_archive('/content/drive/MyDrive/openface_test_task/Detection.tar', format='tar')

Run original training script for 3 epoch to ensure that everything is working.

In [ ]:
%run /content/drive/MyDrive/openface_test_task/LDL/code/train.py



cross_val_index: 0

train      0 | 0.758 | 1.307 | 0.902 | 1.021 |  0 hr 03 min
train      1 | 0.573 | 0.921 | 0.586 | 0.716 |  0 hr 06 min
train      2 | 0.429 | 0.740 | 0.440 | 0.557 |  0 hr 10 min


cross_val_index: 1

train      0 | 0.754 | 1.280 | 0.883 | 1.003 |  0 hr 03 min
train      1 | 0.529 | 0.867 | 0.547 | 0.670 |  0 hr 06 min
train      2 | 0.374 | 0.678 | 0.401 | 0.504 |  0 hr 09 min


cross_val_index: 2

train      0 | 0.760 | 1.270 | 0.876 | 0.999 |  0 hr 03 min
train      1 | 0.548 | 0.875 | 0.553 | 0.680 |  0 hr 06 min
train      2 | 0.426 | 0.709 | 0.424 | 0.539 |  0 hr 09 min


cross_val_index: 3

train      0 | 0.761 | 1.265 | 0.874 | 0.996 |  0 hr 03 min
train      1 | 0.525 | 0.857 | 0.537 | 0.661 |  0 hr 06 min
train      2 | 0.445 | 0.740 | 0.441 | 0.562 |  0 hr 09 min


cross_val_index: 4

train      0 | 0.766 | 1.289 | 0.890 | 1.012 |  0 hr 03 min
train      1 | 0.551 | 0.882 | 0.555 | 0.685 |  0 hr 06 min
train      2 | 0.402 | 0.722 | 0.428 | 0.538 |  0 

In [ ]:
%run /content/drive/MyDrive/openface_test_task/LDL/code/test_doctor.py

            Precision        SE        SP        YI
     class0    0.7310    0.9728    0.7923    0.7651
     class1    0.8864    0.6610    0.9383    0.5993
     class2    0.7234    0.7556    0.9574    0.7129
     class3    0.9773    0.7414    0.9984    0.7398
avg / total    0.8295    0.7827    0.9216    0.7043
              AVE_ACC           0.7943

pre:0.8295 se:0.7827 sp:0.9216 yi:0.7043 acc:0.7943
pre:0.0000 se:0.0000 sp:0.0000 yi:0.0000 acc:0.0000

###########################################
            Precision        SE        SP        YI
     class0    0.7644    0.6411    0.8916    0.5327
     class1    0.6838    0.5672    0.7975    0.3647
     class2    0.3824    0.7065    0.8273    0.5338
     class3    0.6522    0.8182    0.9628    0.7810
avg / total    0.6207    0.6833    0.8698    0.5531
              AVE_ACC           0.6314

pre:0.6207 se:0.6833 sp:0.8698 yi:0.5531 acc:0.6314
pre:0.0000 se:0.0000 sp:0.0000 yi:0.0000 acc:0.0000

##########################################

In [5]:
import os
os.chdir('/content/drive/MyDrive/openface_test_task/LDL/code')

# Hyper Parameters
BATCH_SIZE = 32
BATCH_SIZE_TEST = 20
LR = 0.001              # learning rate
NUM_WORKERS = 12
NUM_CLASSES = 4
sigma=30 * 0.1
lmbd=6 * 0.1

DATA_PATH = '/content/drive/MyDrive/openface_test_task/Classification/JPEGImages/'
TRAIN_FILE = '/content/drive/MyDrive/openface_test_task/Detection/VOC2007/ImageSets/Main/NNEW_trainval_' + '0' + '.txt'
TEST_FILE = '/content/drive/MyDrive/openface_test_task/Detection/VOC2007/ImageSets/Main/NNEW_test_' + '0' + '.txt'

from refactoring.lightning_transfer import ModelRefactoring, DataRefactoring
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from argparse import ArgumentParser

parser = ArgumentParser()

parser.add_argument('--LR', type=float, default=0.001)
parser.add_argument('--sigma', type=float, default=30 * 0.1)
parser.add_argument('--lmbd', type=float, default=6 * 0.1)

args = parser.parse_args(args=[])

parser = Trainer.add_argparse_args(parser)

data = DataRefactoring(BATCH_SIZE, BATCH_SIZE_TEST, NUM_WORKERS, DATA_PATH, TRAIN_FILE, TEST_FILE)

wandb_logger = WandbLogger(name='test_run', project="openface_test_task")
trainer = Trainer(
    logger=wandb_logger,    
    gpus=-1,                
    max_epochs=3)
dict_args = vars(args)
model = ModelRefactoring(**dict_args)

trainer.fit(model, data)


# wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
wandb: Currently logged in as: ecatherina (use `wandb login --relogin` to force relogin)



  | Name | Type   | Params
--------------------------------
0 | cnn  | ResNet | 23.6 M
--------------------------------
23.6 M    Trainable params
53.1 K    Non-trainable params
23.6 M    Total params
94.598    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]